In [ ]:
!pip install datasets

In [1]:
from datasets import load_dataset

ds = load_dataset("Kwaai/IMDB_Sentiment")

In [2]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [3]:
from datasets import concatenate_datasets

# combine train + test a full dataset
full_dataset = concatenate_datasets([ds['train'], ds['test']])  # 50,000

# train dataset（70%）
split1 = full_dataset.train_test_split(test_size=0.3, seed=42)
train = split1['train']       # 70%
temp = split1['test']         # 20%

# validation and test dataset（20%/10%）
split2 = temp.train_test_split(test_size=1/3, seed=42)
validation = split2['train']
test = split2['test']

dataset = {
    "train": train,               # 70%
    "validation": validation,     # 20%
    "test": test                  # 10%
}

print(dataset)

{'train': Dataset({
    features: ['text', 'label'],
    num_rows: 35000
}), 'validation': Dataset({
    features: ['text', 'label'],
    num_rows: 10000
}), 'test': Dataset({
    features: ['text', 'label'],
    num_rows: 5000
})}


In [5]:
import pandas as pd

df = pd.DataFrame(dataset["train"][:10])
print(df)

                                                text  label
0  It's a Time Machine all right. It runs in "rea...      0
1  Recognizing the picture of the diner on the co...      0
2  Hmm, Hip Hop music to a period western. Modern...      0
3  This is a formula B science fiction movie, and...      0
4  Strange yet emotionally disturbing chiller abo...      1
5  As a fan of Eric Rohmer's studies of the conte...      0
6  I am one of Jehovah's Witnesses and I also wor...      1
7  Watching Josh Kornbluth 'act' in this movie re...      0
8  I had fun watching this movie, mainly due to S...      1
9  The penultimate episode of Star Trek's third s...      1


In [ ]:
dataset['train'].to_csv("imdb_train.csv")
dataset['validation'].to_csv("imdb_validation.csv")
dataset['test'].to_csv("imdb_test.csv")